In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
# Sample data
orders = np.array([[1,'orange'], [1,'banana'], [1,'milk'], [2,'banana'], [2,'milk']], dtype=object)

# Generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    
    # For each order, generate a list of items in that order
    for order_id, order_object in groupby(orders, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
    
        # For each item list, generate item pairs, one at a time
        for item_pair in combinations(item_list, 2):
            yield item_pair                                      


# Counter iterates through the item pairs returned by our generator and keeps a tally of their occurrence
Counter(get_item_pairs(orders))

Counter({('orange', 'banana'): 1,
         ('orange', 'milk'): 1,
         ('banana', 'milk'): 2})

In [3]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [4]:
orders = pd.read_csv('order_products__prior.csv')
print('orders -- dimensions: {0};   size: {1}'.format(orders.shape, size(orders)))
display(orders.head())

orders -- dimensions: (32434489, 4);   size: 1037.90 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = orders.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
type(orders)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

pandas.core.series.Series

In [6]:
print('dimensions: {0};   orders_size: {1};   exclusive_orders: {2};   exclusive_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

dimensions: (32434489,);   orders_size: 518.95 MB;   exclusive_orders: 3214874;   exclusive_items: 49677


In [7]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]               

In [12]:
def association_rules(order_item, min_support):

    print("Intial order_item:                                             {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support greater then or equal to min_support {}:     {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item greater then or equal to min_support 0.06: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with more than two items:                               {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item with more than two items:                  {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Result of item pairs frequency and support:          {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support greater than or equal to min_support {}:     {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [13]:
rules = association_rules(orders, 0.06)  

Intial order_item:                                                           32434489
Items with support greater then or equal to min_support 0.06:                2896
Remaining order_item greater then or equal to min_support 0.06:              23520257
Remaining orders with more than two items:                                   2843101
Remaining order_item with more than two items:                               23251234
Result of item pairs frequency and support:                                  6292566
Item pairs with support greater than or equal to min_support 0.06:           3504



In [10]:
# Replace item ID with item name and display association rules
item_name   = pd.read_csv('products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Total 0% Raspberry Yogurt,Fat Free Blueberry Yogurt,1731,0.060884,12118,0.426225,7151,0.251521,0.142845,0.242064,0.567926
3,Blackberry Cucumber Sparkling Water,Kiwi Sandia Sparkling Water,1863,0.065527,11132,0.391544,9097,0.319968,0.167355,0.204793,0.523039
4,Grapefruit Sparkling Water,Lemon Sparkling Water,2164,0.076114,14528,0.510991,9318,0.327741,0.148954,0.232239,0.454487
1,Total 0% Raspberry Yogurt,Fat Free Strawberry Yogurt,1823,0.064120,12118,0.426225,9784,0.344131,0.150437,0.186325,0.437151
5,Icelandic Style Skyr Blueberry Non-fat Yogurt,Non Fat Acai & Mixed Berries Yogurt,2478,0.087158,19224,0.676163,8629,0.303507,0.128901,0.287171,0.424707
6,Non Fat Raspberry Yogurt,Non Fat Acai & Mixed Berries Yogurt,1958,0.068868,16347,0.574971,8629,0.303507,0.119777,0.226909,0.394645
7,Icelandic Style Skyr Blueberry Non-fat Yogurt,Nonfat Icelandic Style Strawberry Yogurt,2699,0.094932,19224,0.676163,10641,0.374274,0.140397,0.253642,0.375119
8,Non Fat Raspberry Yogurt,Nonfat Icelandic Style Strawberry Yogurt,2188,0.076958,16347,0.574971,10641,0.374274,0.133847,0.205620,0.357618
10,Icelandic Style Skyr Blueberry Non-fat Yogurt,Non Fat Raspberry Yogurt,3802,0.133727,19224,0.676163,16347,0.574971,0.197774,0.232581,0.343972
9,Vanilla Skyr Nonfat Yogurt,Nonfat Icelandic Style Strawberry Yogurt,2214,0.077873,18099,0.636594,10641,0.374274,0.122327,0.208063,0.326838
